# 중간 프로젝트: 한국어 뉴스데이터 분류
- 네이버에서 수집한 한국어 뉴스데이터를 가지고 형태소분석 등 전처리를 하고 Pytorch의 FeedForward Neural Networks (FNN)를 통해 그 성능을 살펴보기
- 이 과제의 초점은 한국어 데이터의 전처리 및 여러 딥러닝 모둘에 사용할 수 있는 데이터로 가공하는 것.
- https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_feedforward_neuralnetwork/ 페이지를 근간으로 하며 activation function, 학습률 등은 선택할 수 있음
- 2 hidden layer로 하고 hidden layer의 dimension은 각각 100, 50으로 한다.
- GPU 버전으로 하고, 
- 뉴스 원데이터는 제출할 필요가 없고, 각자의 vocab 화일은 제출해야 함. 실제 채점시에 vocab 만드는 과정 및 관련 선처리 작업은 해당 셀을 실행하지 않고 스킵해도 그 이후의 과정들이 다 실행될 수 있도록 구성할 것.
- 데이터 처리를 위한 필요한 모둘은 다양하게 만들 수 있음. 이를 정교하게 만들어 놓는게 중요
- 이 데이터를 사용해서 앞으로도 파이토치를 활용한 여러 모델을 적용할 계획이므로 전처리, 데이터 변환 등이 잘 이루어질 수 있도록 만들어 놓을 필요
- 제출하는 주피터 노트북 맨 처음에 각자 구현의 특징 전체적으로 설명한 셀을 만들 것.(전처리의 특징, vocab 수, 사용한 형태소 분석기, 사용한 어휘들의 특징, 하이퍼패러미터 값, 그리고 성능 등)
- 과제를 위해 주어진 데이터 외에 프로그램 실행시 필요한 파일들은 과제 제출시 다 첨부해야 함
- 파일 이름 MidTerm_학번.ipynb
- Due 5월 7일 오전 10시 59분 59초까지..

# 각자 구현한 방법론의 특징 및 성능
- 방법
-전처리
- vocab 특징
- 성능 등...

## 뉴스 데이터를  pandas로 부르기
- 첨부된 newsdata.zip은 네이버 뉴스 기사를 모은 것으로 0에서 7까지 레이블이 되어 있다.
    - 정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7) 
- 편의를 위해 이 데이터를 tsv화일로 정리한 것이 newsdata.tsv이다.     
- 이 tsv 파일은 filename, headline, body, label로 되어 있으며 headline은 기사의 제목이며, body는 실제 기사이다.
- 각 범주의 파일은 200개로 되어 있으며 파일네임은 각 범주 0-7에서 시작하며 각범주의 끝은 X199NewsData.txt로 되어 있다.
- 각 범주 200개의 파일 중에서 160개는 학습자료로 나머지 40개는 시험자료로 삼는다. 시험자료는 각 디렉토리의 x160 - x199로 시작하는 파일로 한다.
- 헤드라인과 뉴스본문(body)를 합쳐서 자료로 사용할 수 있다.
- newline표시가 \n\n으로 되어 있다.

In [2]:
# 코드 작성

import pandas as pd
news = pd.read_table("newsdata.tsv", sep = '\t')

## 불러온 파일 체크
#news.head(20)
news.tail(20)

,filename,headline,body,label
1580,7180NewsData.txt,19일부터 전국 BMX(자전거 모터크로스)대회 열린다,대한자전거연맹이 오는 19~20일 세종 BMX경기장에서 '2018 제3회 위아위스배...,7
1581,7181NewsData.txt,"첨단 냉각요법 '크라이오', 월드컵, 아시안게임 등 앞두고 주목",2018년은 유독 전세계적인 스포츠 이벤트가 많은 해다. 평창 동계올림픽을 시작으로...,7
1582,7182NewsData.txt,"야외에서 볼링을? 부산컵, 국제오픈 화제",야외 레인에서 펼쳐지는 볼링대회가 부산에서 열립니다.\n\n한국프로볼링협회(PBA)...,7
1583,7183NewsData.txt,경기실적 및 경기동영상 등록관리 지원 사업 설명회,"【서울=뉴시스】 문성대 기자 = 대한체육회기 17, 21일 서울올림픽파크텔 아테네홀...",7
1584,7184NewsData.txt,"KBF, 7월 전국당구대회로 UMB-코줌대회에 ‘맞불’",‘7월 연맹회장배 당구대회 개최 일정’ 홈피에 공지\nUMB-코줌 ‘3쿠션마스터즈챌...,7
1585,7185NewsData.txt,"한일장신대 김정민, 전국양궁선수권 컴파운드 2관왕",(완주=뉴스1) 임충식 기자 = 한일장신대학교 김정민이 ‘제52회 전국남여양궁종별선...,7
1586,7186NewsData.txt,한국 킥복싱 2018 亞선수권 금1 은3 동6…16일 개선,[헤럴드경제=조용직 기자] 대한민국 킥복싱이 아시아 40여개국 300여 명의 선수들...,7
1587,7187NewsData.txt,"[인천챌린저]한나래, 두 시간 넘는 혈투 끝에 2회전 진출","[테니스코리아= 이은미 기자]'디펜딩 챔피언' 한나래(인천시청, 269위)가 인천챌...",7
1588,7188NewsData.txt,[3쿠션 월드컵] 21일부터 베트남 호치민에서 개최,[엠스플뉴스]\n \n2018년 두 번째 3쿠션 월드컵 대회가 베트남 호치민에서 5...,7
1589,7189NewsData.txt,"부산오픈 챌린저, 16일 경기 우천으로 연기",[테니스코리아= (부산)박준용 기자]5월 16일 부산 스포원 테니스장에서 열릴 예정...,7


## 전처리를 위한 한국어 형태소 분석기 사용
- 신문기사 자료는 특수문자, 괄호표현, 한자 등 필요없는 경우가 많아서 이를 제거할 필요
- 이를 처리하는 것이 매우 중요한데, 이것을 연습.
- 정규표현을 사용할 수도 있고, 한국어 형태소 분석기를 사용할 수 있음
- 한국어 형태소 분석기는 KoNLPy (http://konlpy.org/ko/v0.4.4/ )에서 다운로드 받을 수 있음
- 이 페이지에 있는 각 형태소 분석기 특징을 잘 파악하여 본인이 원하는 형태소 분석기를 사용


### 참고: 본 기사에 대한 각 형태소 분석기 특징



#### Kkma

오류 예시: \['남비난', '비난', '미정상회담', '정상', '회담', '맥스', '맥스선더', '태영호', '영호', '17', '11', '11일', '2주간', '주간'\]

특징: 중복 처리되는 단어들 많음, 숫자와 날짜 남아있음 (twitter, komoran은 제거함)


#### Hannanum

오류 예시: \['【서울=뉴시스】김성진', '22일'\]

특징: 일부 특수문자 처리 안됨, 숫자와 날짜 남아있음 (twitter, komoran은 제거함)


#### Twitter

예시: \['연일', '남비', '한미정상회담', '경고', '메시지', '발신', '맥스', '선더', '영호', '불만', '표시', '최종', '조율', '의제', '반발', '북중', '관계', '개선', '시진핑', '의중', '반영', '서울', '뉴시스', '김성진', '기자', '북한', '이틀', '대남', '대미', '비난', '의도', '대해', '관심', '북한', '물론', '대미', '관계', '조금', '무게', '모양새', '한미', '정상회담', '비난', '수위', '한국', '미국', '각각', '불만', '풀이', '북한', '지난', '한미', '공군', '연합', '훈련', '맥스', '선더', '자신', '최고', '존엄', '영호', '주영', '북한', '대사관', '공사', '대해', '불만', '출하', '남북', '고위', '회담', '무기한', '연기', '통보', '김계관', '외무성', '부상', '담화', '통해', '포기', '보상', '검증', '비핵화', '미국', '리선', '조국', '평화통일', '위원회', '위원장', '조선', '중앙', '통신', '통해', '남조선', '당국', '괴이', '논리', '조선반도', '평화', '화해', '흐름', '가로막', '장애물', '하나', '북침', '전쟁', '연습', '합리화', '비방', '중상', '지속', '철면피', '파렴치', '극치', '비난', '위원장', '북남', '고위', '회담', '중지', '엄중', '사태', '남조선', '정권', '다시', '마주', '라며', '차후', '북남관계', '방향', '전적', '남조선', '당국', '행동', '리선', '위원장', '발언', '표면', '판문점', '선언', '정신', '부합', '즉각', '회담', '우리', '정부', '성명', '대한', '반발', '전날', '맥스', '선더', '훈련', '공사', '대해', '경고', '메시지', '발신', '방치', '정부', '대한', '유감', '불만', '표시', '전문가', '북한', '일련', '행보', '최종', '비핵화', '담판', '벌이', '미국', '방점', '홍민', '통일', '연구원', '북한', '연구실', '북한', '최종', '미국', '라며', '미국', '조절', '의제', '방식', '문제', '제기', '실장', '자신', '통제', '이야기', '최대한', '폼페이', '조율', '이야기', '표현', '라며', '자신', '선의', '비핵화', '취지', '왜곡', '때문', '정도', '정리', '입장', '다음주', '개최', '한미', '정상회담', '목표', '비난', '수위', '가능성', '제기', '자칫', '한미', '구도', '시도', '분석', '지난', '북중', '정상회담', '이후', '북중', '관계', '중국', '의중', '가능성', '제기', '지난', '김정은', '북한', '국무위원', '중국', '외교부', '평양', '면담', '중국', '다롄', '시진핑', '중국', '국가주석', '정상회담', '지난', '주석', '도널드', '트럼프', '미국', '대통령', '전화통화', '미국', '북한', '합리', '안보', '우려', '바란', '기도', '중국', '북한', '모두', '견제', '주한', '미군', '군사', '동맹', '미국', '핵우산', '제공', '염두', '발언', '트럼프', '대통령', '백악관', '옌스', '스톨', '베르크', '북대서양', '조약', '기구', '사무', '총장', '만난', '자리', '주석', '김정은', '영향', '가능성', '매우', '면서', '무슨', '일이', '기도', '신범철', '아산', '정책', '연구원', '안보', '통일', '센터', '북중', '관계', '정세', '변화', '따라서', '북한', '입장', '약간', '비핵화', '협상', '미국', '강도', '북한', '나름', '목소리', '위해', '남북관계', '역시', '자신', '불만', '다만', '북한', '협상', '위해', '행보', '풀이', '한편', '경색', '국면', '남북', '대화', '북한', '지난', '주간', '계획', '맥스', '선더', '훈련', '명분', '남북', '고위', '회담', '만큼', '한미', '정상회담', '맥스', '선더', '훈련', '종료', '다시', '재개', '관측', '이후', '바로', '북미', '정상회담', '때문', '판문점', '선언', '개최', '합의', '남북', '장성', '회담', '개최', '전망'\]


특징: 속도는 빠르나, komoran에 비해 분석되는 어휘 수가 적음


#### Komoran

예시: \['연일', '비난', '한미정상회담', '경고', '메시지', '발신', '맥스', '선더', '영호', '불만', '표시', '의제', '반발', '북중', '관계', '개선', '시진핑', '의중', '반영', '서울', '뉴시스', '김성진', '기자', '북한', '이틀', '대남', '대미', '비난', '의도', '관심', '북한', '대미', '관계', '무게', '모양새', '한미 정상회담', '비난', '수위', '한국', '미국', '불만', '표출', '풀이', '북한', '미 공군', '연합', '훈련', '맥스', '선더', '자신', '최고', '존엄', '영호', '주영', '북한', '대사관', '공사', '불만', '표출', '남북', '고위급', '회담', '무기', '연기', '통보', '김계관', '외무성', '부상', '담화', '포기', '보상', '완전', '검증', '비핵화', '주장', '미국', '규탄', '조국평화통일위원회', '위원장', '조선중앙통신', '남조선', '당국', '논리', '조선반도', '평화', '화해', '흐름', '장애물', '제거', '전쟁', '연습', '합리', '비방', '중상', '지속', '철면피', '파렴치', '극치', '비난', '위원장', '북남', '고위급', '회담', '중지', '사태', '해결', '남조선', '정권', '차후', '북남', '관계', '방향', '전적', '남조선', '당국', '행동', '여하', '위원장', '발언', '표면', '판문점', '선언', '정신', '부합', '회담', '정부', '성명', '반발', '전날', '맥스', '선더', '훈련', '공사', '경고', '메시지', '발신', '방치', '정부', '유감', '불만', '표시', '전문가', '북한', '일련', '행보', '결국', '최종', '비핵화', '담판', '미국', '방점', '분석', '통일연구원', '북한', '연구실장', '북한', '최종', '미국', '미국', '조절', '의제', '방식', '문제', '제기', '분석', '실장', '자신', '통제', '이야기', '최대한', '폼페이', '오와', '조율', '이야기', '불편', '표현', '자신', '선의', '비핵화', '취지', '왜곡', '불편', '때문', '정도', '정리', '입장', '다음', '개최', '한미 정상회담', '목표', '비난', '수위', '제기', '한미', '구도', '시도', '분석', '북중', '정상회담', '이후', '북중', '관계', '개선', '중국', '의중', '반영', '제기', '김정은', '북한', '국무', '위원장', '중국', '외교부장', '평양', '면담', '중국', '다롄', '중국 국가주석', '정상회담', '주석', '도널드 트럼프', '미국 대통령', '전화통', '미국', '북한', '합리', '안보', '우려', '고려', '중국', '북한', '양측', '견제', '주한미군', '군사', '동맹', '미국', '핵우산', '제공', '염두', '발언', '트럼프', '대통령', '백악관', '옌스', '스톨', '베르크', '북대서양조약기구', '사무총장', '자리', '주석', '김정은', '영향', '신범철', '아산정책연구원', '안보', '통일', '센터', '북중', '관계', '정세', '변화', '북한', '입장', '비핵화', '협상', '미국', '강도', '북한', '나름', '목소리', '이야기', '남북관계', '자신', '불만', '표현', '북한', '협상', '전개', '행보', '풀이', '한편', '경색', '국면', '남북', '대화', '북한', '주간', '계획', '맥스', '선더', '훈련', '명분', '남북', '고위급', '회담', '연기', '만큼', '한미 정상회담', '맥스', '선더', '훈련', '종료', '재개', '관측', '이후', '북미', '정상회담', '때문', '판문점', '선언', '개최', '합의', '남북', '장성', '회담', '개최', '전망'\]

특징: 4개 모듈 중에서는 정확도가 높다고 판단. '조국평화통일위원회'와 같은 복함 단어를 개별 토큰으로 분석.

### 전처리에서 주로 해야할 것 들(참고사항이며 필요에 따라 추가할 수 있음)


1. 특수문자는 '…'과 '·'만 제거. 나머지는 konlpy에서 처리.
2. 괄호 안에 있는 정보는 부가적인 내용으로 판단되어 제거. (부가설명, 이름, 나이, 한자 병기 등)
3. 2번 처리 후 남아있는 한자들은 한글로 번역.
   - 한자 변환모듈은 https://pypi.org/project/hanja/ 참조하여 모듈을 사용할 수 있음
   - hanja.translate('大韓民國은 民主共和國이다.', 'substitution')
     --> '대한민국은 민주공화국이다.'
4. konlpy 모듈로 형태소 분석 한 후 어떤 품사를 사용할지 자율적 결정 (명사, 동사, 형용사 등...)
5. 중복되는 어휘, 1음절 토큰은 제거.
6. 폴더 안에 있는 모든 기사들을 토큰화 하고, 최종적인 VOCAB 사전으로 저장. (단어 : 인덱스) 2회 미만 출현 단어들은 무시.



In [6]:
!pip3 install hanja

Processing /Users/jonghokim/Library/Caches/pip/wheels/f6/8f/43/239c7c311f1ad8a6258fe189d56ff4087a63a1c7b1ca9036f6/hanja-0.13.2-py3-none-any.whl
Processing /Users/jonghokim/Library/Caches/pip/wheels/23/b9/73/57aaccb6957d94ed63f474b51a9f7f992c5eff4635052c0557/PyYAML-5.1.2-cp37-cp37m-macosx_10_9_x86_64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.3.1
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [5]:
#필요한 모듈 부르기
# 몇 개만 나열한 것. 필요에 따라서 지우고 새로운 것 추가할 필요

import sys
from os import listdir
import time
import re
import numpy as np
import codecs
import collections
import hanja
from konlpy.tag import Twitter
from konlpy.tag import Komoran
from konlpy.tag import Kkma

import torch
import torch.nn as nn
import pandas as pd

ModuleNotFoundError: No module named 'hanja'

In [ ]:
# 원데이터를 가공해서 pd dataframe으로 읽을 수 있도록 할 것 (tsv)

In [ ]:
## 전처리 및 파일을 FNN에 넣기 위한 필요한 모듈 설정

In [ ]:
## 전처리 및 vocab 만들기

In [ ]:
## encoding, padding, one-hot encoding ...

In [ ]:
## MAKING DATASET ITERABLE
'''

batch_size = 128
n_iters = 100
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = 
test_loader = 


In [ ]:
## Create Model Class

class FFN(nn.Module):
    def __init__( ):
        nn.Linear 사용
        
    def forward( ):
        

In [ ]:
## INSTANTIATE MODEL CLASS
'''
input_dim = 
hidden1_dim = 100
hidden2_dim = 50
output_dim = 

model = 


In [ ]:
## INSTANTIATE LOSS CLASS

criterion = 


In [ ]:
## INSTANTIATE OPTIMIZER CLASS

learning_rate = 

optimizer = 


In [ ]:
## TRAIN THE MODEL

iter = 0
for epoch in range(num_epochs):
   
           코드

        if iter % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
           
           코드 
            
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))